In [ ]:
!wget -N https://rl-learning-storage.s3.amazonaws.com/asteroids.zip
!unzip -o asteroids.zip
!pip install gym-retro
#!pip3 install pyglet==1.5.0 gym-retro
!python3 -m retro.import ./
!pip install keras-rl
!pip install tensorflow==1.14

In [ ]:
pip install wrapt --upgrade --ignore-installed
pip install tensorflow==1.14
pip install keras-rl

In [ ]:
import os
import shutil
path = '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/retro/data/stable/Asteroids-Atari2600'

directory_contents = os.listdir(path)
print(directory_contents)

for item in directory_contents:
    if os.path.isdir(item):
        print(item)

In [ ]:
os.remove('/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/retro/data/stable/Asteroids-Atari2600/scenario.json')
shutil.move('/home/ec2-user/SageMaker/scenario.json', '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/retro/data/stable/Asteroids-Atari2600/scenario.json')

In [1]:
import numpy as np
import gym
import retro
#Create Environment for Asteroid
ENV_NAME = 'Asteroids-Atari2600'
env = retro.make(game=ENV_NAME, record='.', use_restricted_actions=retro.Actions.DISCRETE)
nb_actions = env.action_space.n

In [2]:
#Check Game Properties
print(env.buttons)
print(env.action_space)
print(env.observation_space)
state_shape = env.observation_space.shape
state_shape[0]

['BUTTON', None, 'SELECT', 'RESET', 'UP', 'DOWN', 'LEFT', 'RIGHT']
Discrete(18)
Box(210, 160, 3)


210

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Convolution3D, MaxPooling2D, InputLayer
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.agents import SARSAAgent,DDPGAgent
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy,BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
#Determine the structure of neural net
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(420))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 420)               42336420  
_________________________________________________________________
activation_1 (Activation)    (None, 420)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 18)                7578      
_________________________________________________________________
activation_2 (Activation)    (None, 18)                0         
Total params: 42,343,998
Trainable params: 42,343,998
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Decide Policy Memory for agent
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,nb_steps=500)
memory = SequentialMemory(limit=100000000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, gamma=0.99, nb_steps_warmup=1000,target_model_update=1000, policy=policy,enable_double_dqn=True,train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

# Train the model 
steps = 50000
max_reward = 0

def build_callbacks(env_name):
    checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
    log_filename = 'dqn_{}_log.json'.format(env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=1000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

callbacks = build_callbacks(ENV_NAME)
dqn.fit(env, nb_steps=steps, visualize=False,verbose=2,callbacks=callbacks,log_interval=1000)
dqn.save_weights('dqn_' + ENV_NAME + '_weights_{step}.h5f', overwrite=True)
test = dqn.test(env, nb_episodes = 3, visualize=False)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training for 50000 steps ...

  2183/50000: episode: 1, duration: 328.135s, episode steps: 2183, steps per second: 7, episode reward: -63999006.000, mean reward: -29316.998 [-15999984.000, 100.000], mean action: 6.535 [0.000, 17.000], mean observation: 1.824 [0.000, 240.000], loss: 22057.419435, mae: 1235.783317, mean_q: 29.434680, mean_eps: 0.100000
  3822/50000: episode: 2, duration: 481.445s, episode steps: 1639, steps per second: 3, episode reward: -63999356.000, mean reward: -39047.807 [-15999984.000, 100.000], mean action: 9.126 [0.000, 17.000], mean observation: 2.266 [0.000, 240.000], loss: 45126.480889, mae: 2511.957607, mean_q: 10.463821, mean_eps: 0.100000
  5427/50000: episode: 3, duration: 522.614s, episode steps: 1605, steps per second: 3, episode reward: -63999506.000, mean reward: -39875.082 [-15999984.000, 100.000], mean action: 9.897 [0.000, 17.000], mean observation: 2.088 [

In [ ]:
env.close()

In [ ]:
import os
import boto3
import re
import copy
import time
import io
import struct
from time import gmtime, strftime
from sagemaker import get_execution_role

region = boto3.Session().region_name
bucket='rl-learning-storage' # Replace with your s3 bucket name
prefix = 'RL-Information' # Used as part of the path in the bucket where you store data
url = 's3://{}/{}'.format(bucket, prefix)
boto3.Session().resource('s3').Bucket(bucket).Object(prefix).upload_file('*.h5f')
boto3.Session().resource('s3').Bucket(bucket).Object(prefix).upload_file('*.bk2')
boto3.Session().resource('s3').Bucket(bucket).Object(prefix).upload_file('*.json')
print('Done writing to {}'.format(url))